In [1]:
import numpy as np
import pandas as pd

In [2]:
data_preprocessed=pd.read_csv('C:/Users/anil/Downloads/Absenteeism_preprocessed.csv')

In [3]:
data_preprocessed.head()

,Reason_1,Reason_2,Reason_3,Reason_4,Date,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours,Month_value,Day of the Week
0,0,0,0,1,2015-07-07,289,36,33,239.554,30,0,2,1,4,7,1
1,0,0,0,0,2015-07-14,118,13,50,239.554,31,0,1,0,0,7,1
2,0,0,0,1,2015-07-15,179,51,38,239.554,31,0,0,0,2,7,2
3,1,0,0,0,2015-07-16,279,5,39,239.554,24,0,2,0,4,7,3
4,0,0,0,1,2015-07-23,289,36,33,239.554,30,0,2,1,2,7,3


In [4]:
#we are using logistic regression as <3=less absent ,>3=more absent
data_preprocessed['Absenteeism Time in Hours'].median()

3.0

In [5]:
targets=np.where(data_preprocessed['Absenteeism Time in Hours']>3,1,0)

In [6]:
len(targets)

700

In [7]:
data_preprocessed['Excessive Absenteesim']=targets

In [8]:
data_preprocessed.head()

,Reason_1,Reason_2,Reason_3,Reason_4,Date,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours,Month_value,Day of the Week,Excessive Absenteesim
0,0,0,0,1,2015-07-07,289,36,33,239.554,30,0,2,1,4,7,1,1
1,0,0,0,0,2015-07-14,118,13,50,239.554,31,0,1,0,0,7,1,0
2,0,0,0,1,2015-07-15,179,51,38,239.554,31,0,0,0,2,7,2,0
3,1,0,0,0,2015-07-16,279,5,39,239.554,24,0,2,0,4,7,3,1
4,0,0,0,1,2015-07-23,289,36,33,239.554,30,0,2,1,2,7,3,0


In [9]:
targets.shape

(700,)

In [10]:
#getting % of 1's
print(((targets.sum()/targets.shape[0])*100).round(2),'%')


45.57 %


In [11]:
data_with_targets=data_preprocessed.drop(['Absenteeism Time in Hours','Daily Work Load Average','Day of the Week','Distance to Work'],axis=1)

In [12]:
data_with_targets 
#dropping daily wLA,DOW,DW because of low contribution to model

,Reason_1,Reason_2,Reason_3,Reason_4,Date,Transportation Expense,Age,Body Mass Index,Education,Children,Pets,Month_value,Excessive Absenteesim
0,0,0,0,1,2015-07-07,289,33,30,0,2,1,7,1
1,0,0,0,0,2015-07-14,118,50,31,0,1,0,7,0
2,0,0,0,1,2015-07-15,179,38,31,0,0,0,7,0
3,1,0,0,0,2015-07-16,279,39,24,0,2,0,7,1
4,0,0,0,1,2015-07-23,289,33,30,0,2,1,7,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
695,1,0,0,0,2018-05-23,179,40,22,1,2,0,5,1
696,1,0,0,0,2018-05-23,225,28,24,0,1,2,5,0
697,1,0,0,0,2018-05-24,330,28,25,1,0,0,5,1
698,0,0,0,1,2018-05-24,235,32,25,1,0,0,5,0


In [13]:
#data_with_targets.loc[:,'Reason_1':'Day of the Week'] same as
data_with_targets.iloc[:,0:12].head()

,Reason_1,Reason_2,Reason_3,Reason_4,Date,Transportation Expense,Age,Body Mass Index,Education,Children,Pets,Month_value
0,0,0,0,1,2015-07-07,289,33,30,0,2,1,7
1,0,0,0,0,2015-07-14,118,50,31,0,1,0,7
2,0,0,0,1,2015-07-15,179,38,31,0,0,0,7
3,1,0,0,0,2015-07-16,279,39,24,0,2,0,7
4,0,0,0,1,2015-07-23,289,33,30,0,2,1,7


In [14]:
unscaled_inputs=data_with_targets.iloc[:,0:12]
#date error so dropping the date is necessary
unscaled_inputs=unscaled_inputs.drop(['Date'],axis=1)

In [15]:
from sklearn.preprocessing import StandardScaler

In [16]:
absenteesim_scaler=StandardScaler()

In [17]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler

# create the Custom Scaler class

class CustomScaler(BaseEstimator,TransformerMixin): 
    
    # init or what information we need to declare a CustomScaler object
    # and what is calculated/declared as we do
    
    def __init__(self,columns,copy=True,with_mean=True,with_std=True):
        
        # scaler is nothing but a Standard Scaler object
        self.scaler = StandardScaler(copy,with_mean,with_std)
        # with some columns 'twist'
        self.columns = columns
        self.mean_ = None
        self.var_ = None
        
    
    # the fit method, which, again based on StandardScale
    
    def fit(self, X, y=None):
        self.scaler.fit(X[self.columns], y)
        self.mean_ = np.mean(X[self.columns])
        self.var_ = np.var(X[self.columns])
        return self
    
    # the transform method which does the actual scaling

    def transform(self, X, y=None, copy=None):
        
        # record the initial order of the columns
        init_col_order = X.columns
        
        # scale all features that you chose when creating the instance of the class
        X_scaled = pd.DataFrame(self.scaler.transform(X[self.columns]), columns=self.columns)
        
        # declare a variable containing all information that was not scaled
        X_not_scaled = X.loc[:,~X.columns.isin(self.columns)]
        
        # return a data frame which contains all scaled features and all 'not scaled' features
        # use the original order (that you recorded in the beginning)
        return pd.concat([X_not_scaled, X_scaled], axis=1)[init_col_order]

In [18]:
unscaled_inputs.columns.values

array(['Reason_1', 'Reason_2', 'Reason_3', 'Reason_4',
       'Transportation Expense', 'Age', 'Body Mass Index', 'Education',
       'Children', 'Pets', 'Month_value'], dtype=object)

In [19]:
#columns_to_scale=['Transportation Expense', 'Distance to Work', 'Age','Daily Work Load Average', 'Body Mass Index','Children', 'Pets', 'Month_value', 'Day of the Week']
columns_to_omit=['Reason_1','Reason_2','Reason_3','Reason_4','Education']

In [20]:
columns_to_scale = [x for x in unscaled_inputs.columns.values if x not in columns_to_omit]

In [21]:
absenteesim_scaler=CustomScaler(columns_to_scale)

In [22]:
absenteesim_scaler.fit(unscaled_inputs)

C:\Users\anil\anaconda3\lib\site-packages\sklearn\base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


CustomScaler(columns=['Transportation Expense', 'Age', 'Body Mass Index',
                      'Children', 'Pets', 'Month_value'],
             copy=None, with_mean=None, with_std=None)

In [23]:
scaled_inputs=absenteesim_scaler.transform(unscaled_inputs)

In [24]:
sdf=pd.DataFrame(scaled_inputs)

In [25]:
pd.options.display.max_columns=None
pd.options.display.max_rows=None
display(sdf)

,Reason_1,Reason_2,Reason_3,Reason_4,Transportation Expense,Age,Body Mass Index,Education,Children,Pets,Month_value
0,0,0,0,1,1.005844,-0.536062,0.767431,0,0.880469,0.268487,0.030796
1,0,0,0,0,-1.574681,2.130803,1.002633,0,-0.019280,-0.589690,0.030796
2,0,0,0,1,-0.654143,0.248310,1.002633,0,-0.919030,-0.589690,0.030796
3,1,0,0,0,0.854936,0.405184,-0.643782,0,0.880469,-0.589690,0.030796
4,0,0,0,1,1.005844,-0.536062,0.767431,0,0.880469,0.268487,0.030796
5,0,0,0,1,-0.654143,0.248310,1.002633,0,-0.919030,-0.589690,0.929019
6,0,0,0,1,2.092381,-1.320435,0.061825,0,-0.019280,2.843016,0.030796
7,0,0,0,1,0.568211,-0.065439,-0.878984,0,2.679969,-0.589690,0.030796
8,0,0,1,0,-1.016322,-0.379188,-0.408580,0,0.880469,-0.589690,-0.268611
9,0,0,0,1,0.190942,0.091435,0.532229,1,-0.019280,0.268487,0.030796


In [26]:
from sklearn.model_selection import train_test_split

In [27]:
x_train,x_test,y_train,y_test=train_test_split(scaled_inputs,targets,test_size=0.2,random_state=20)

In [28]:
print(x_train.shape,x_test.shape,y_train.shape,y_test.shape)

(560, 11) (140, 11) (560,) (140,)


In [29]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [30]:
reg=LogisticRegression()

In [31]:
#trainig the model
reg.fit(x_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [32]:
reg.score(x_train,y_train)

0.7767857142857143

### MANUALLY finding accuracy

In [33]:
model_outputs=reg.predict(x_train)

In [34]:
model_outputs

array([0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0,
       0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1,
       0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0,
       0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0,
       0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0,
       1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1,
       1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1,
       0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1,
       0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0,

In [35]:
y_train

array([0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0,
       1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1,
       1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1,
       1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1,
       0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1,
       1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0,

In [36]:
model_outputs==y_train

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True, False,  True, False, False,  True,  True,  True,  True,
       False,  True, False,  True, False, False,  True,  True,  True,
       False,  True,  True,  True,  True,  True,  True,  True,  True,
       False, False, False, False,  True,  True,  True,  True, False,
        True,  True,  True,  True,  True, False,  True,  True,  True,
        True,  True,  True,  True,  True, False,  True,  True,  True,
        True,  True,  True,  True,  True,  True, False,  True,  True,
        True,  True,  True, False,  True,  True,  True,  True,  True,
       False,  True, False,  True,  True, False, False, False,  True,
        True,  True,  True,  True,  True,  True,  True, False,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True, False,  True,  True,  True,  True,
       False,  True,  True,  True,  True, False,  True,  True,  True,
        True,  True,

In [37]:
np.sum((model_outputs==y_train))

435

In [38]:
model_outputs.shape[0]

560

In [39]:
print(np.sum((model_outputs==y_train))/model_outputs.shape[0])

0.7767857142857143


In [40]:
scaled_inputs

,Reason_1,Reason_2,Reason_3,Reason_4,Transportation Expense,Age,Body Mass Index,Education,Children,Pets,Month_value
0,0,0,0,1,1.005844,-0.536062,0.767431,0,0.880469,0.268487,0.030796
1,0,0,0,0,-1.574681,2.130803,1.002633,0,-0.019280,-0.589690,0.030796
2,0,0,0,1,-0.654143,0.248310,1.002633,0,-0.919030,-0.589690,0.030796
3,1,0,0,0,0.854936,0.405184,-0.643782,0,0.880469,-0.589690,0.030796
4,0,0,0,1,1.005844,-0.536062,0.767431,0,0.880469,0.268487,0.030796
5,0,0,0,1,-0.654143,0.248310,1.002633,0,-0.919030,-0.589690,0.929019
6,0,0,0,1,2.092381,-1.320435,0.061825,0,-0.019280,2.843016,0.030796
7,0,0,0,1,0.568211,-0.065439,-0.878984,0,2.679969,-0.589690,0.030796
8,0,0,1,0,-1.016322,-0.379188,-0.408580,0,0.880469,-0.589690,-0.268611
9,0,0,0,1,0.190942,0.091435,0.532229,1,-0.019280,0.268487,0.030796


In [41]:
sdf.columns=unscaled_inputs.columns

In [42]:
sdf

,Reason_1,Reason_2,Reason_3,Reason_4,Transportation Expense,Age,Body Mass Index,Education,Children,Pets,Month_value
0,0,0,0,1,1.005844,-0.536062,0.767431,0,0.880469,0.268487,0.030796
1,0,0,0,0,-1.574681,2.130803,1.002633,0,-0.019280,-0.589690,0.030796
2,0,0,0,1,-0.654143,0.248310,1.002633,0,-0.919030,-0.589690,0.030796
3,1,0,0,0,0.854936,0.405184,-0.643782,0,0.880469,-0.589690,0.030796
4,0,0,0,1,1.005844,-0.536062,0.767431,0,0.880469,0.268487,0.030796
5,0,0,0,1,-0.654143,0.248310,1.002633,0,-0.919030,-0.589690,0.929019
6,0,0,0,1,2.092381,-1.320435,0.061825,0,-0.019280,2.843016,0.030796
7,0,0,0,1,0.568211,-0.065439,-0.878984,0,2.679969,-0.589690,0.030796
8,0,0,1,0,-1.016322,-0.379188,-0.408580,0,0.880469,-0.589690,-0.268611
9,0,0,0,1,0.190942,0.091435,0.532229,1,-0.019280,0.268487,0.030796


In [43]:
reg.intercept_

array([-1.60957471])

In [44]:
reg.coef_

array([[ 2.77151176,  0.93168817,  3.09210221,  0.8090592 ,  0.62505482,
        -0.17390339,  0.28829409, -0.24081615,  0.35753531, -0.27337422,
         0.00781237]])

In [45]:
summary_table=pd.DataFrame(columns=['Features'],data=sdf.columns.values)
summary_table['Coefficients']=np.transpose(reg.coef_)
summary_table

,Features,Coefficients
0,Reason_1,2.771512
1,Reason_2,0.931688
2,Reason_3,3.092102
3,Reason_4,0.809059
4,Transportation Expense,0.625055
5,Age,-0.173903
6,Body Mass Index,0.288294
7,Education,-0.240816
8,Children,0.357535
9,Pets,-0.273374


In [46]:
#adding intercept
summary_table.index=summary_table.index+1
summary_table.loc[0]=['Intercept',reg.intercept_[0]]
summary_table=summary_table.sort_index()

In [47]:
summary_table

,Features,Coefficients
0,Intercept,-1.609575
1,Reason_1,2.771512
2,Reason_2,0.931688
3,Reason_3,3.092102
4,Reason_4,0.809059
5,Transportation Expense,0.625055
6,Age,-0.173903
7,Body Mass Index,0.288294
8,Education,-0.240816
9,Children,0.357535


In [48]:
summary_table['odds_ratio']=np.exp(summary_table['Coefficients'])

In [49]:
summary_table

,Features,Coefficients,odds_ratio
0,Intercept,-1.609575,0.199973
1,Reason_1,2.771512,15.982778
2,Reason_2,0.931688,2.538791
3,Reason_3,3.092102,22.023327
4,Reason_4,0.809059,2.245794
5,Transportation Expense,0.625055,1.868348
6,Age,-0.173903,0.840378
7,Body Mass Index,0.288294,1.334150
8,Education,-0.240816,0.785986
9,Children,0.357535,1.429801


In [50]:
summary_table=summary_table.sort_values('odds_ratio',ascending=False)

In [51]:
summary_table

,Features,Coefficients,odds_ratio
3,Reason_3,3.092102,22.023327
1,Reason_1,2.771512,15.982778
2,Reason_2,0.931688,2.538791
4,Reason_4,0.809059,2.245794
5,Transportation Expense,0.625055,1.868348
9,Children,0.357535,1.429801
7,Body Mass Index,0.288294,1.334150
11,Month_value,0.007812,1.007843
6,Age,-0.173903,0.840378
8,Education,-0.240816,0.785986


In [52]:
#OBSERVATION
#A feature is not important
#if coeff is around 0
#if odds ratio is around 1

### Testing the model

In [53]:
reg.score(x_test,y_test)

0.7357142857142858

In [54]:
predicted_proba=reg.predict_proba(x_test)
predicted_proba

array([[0.75466347, 0.24533653],
       [0.60917591, 0.39082409],
       [0.48329093, 0.51670907],
       [0.75768232, 0.24231768],
       [0.08357741, 0.91642259],
       [0.3052464 , 0.6947536 ],
       [0.303675  , 0.696325  ],
       [0.11636888, 0.88363112],
       [0.7400284 , 0.2599716 ],
       [0.75596036, 0.24403964],
       [0.50609784, 0.49390216],
       [0.19501503, 0.80498497],
       [0.06248668, 0.93751332],
       [0.7055465 , 0.2944535 ],
       [0.29675526, 0.70324474],
       [0.52028649, 0.47971351],
       [0.50551315, 0.49448685],
       [0.50843643, 0.49156357],
       [0.36713074, 0.63286926],
       [0.06422143, 0.93577857],
       [0.73822433, 0.26177567],
       [0.75768232, 0.24231768],
       [0.47994423, 0.52005577],
       [0.47760936, 0.52239064],
       [0.22619725, 0.77380275],
       [0.74047815, 0.25952185],
       [0.51148533, 0.48851467],
       [0.87702735, 0.12297265],
       [0.24005377, 0.75994623],
       [0.75768232, 0.24231768],
       [0.

In [55]:
#column0:probability that a given individual is NOT expected to be absent from work for more than 3 hours
#column1:probability that a given individual is expected to be absent from work for more than 3 hours
predicted_proba[:,1]

array([0.24533653, 0.39082409, 0.51670907, 0.24231768, 0.91642259,
       0.6947536 , 0.696325  , 0.88363112, 0.2599716 , 0.24403964,
       0.49390216, 0.80498497, 0.93751332, 0.2944535 , 0.70324474,
       0.47971351, 0.49448685, 0.49156357, 0.63286926, 0.93577857,
       0.26177567, 0.24231768, 0.52005577, 0.52239064, 0.77380275,
       0.25952185, 0.48851467, 0.12297265, 0.75994623, 0.24231768,
       0.38859882, 0.71238179, 0.69821485, 0.49507156, 0.24231768,
       0.59772596, 0.26042186, 0.78022686, 0.4398591 , 0.60641126,
       0.24188848, 0.49713003, 0.25862385, 0.40689715, 0.80759194,
       0.59889345, 0.71944702, 0.24317762, 0.24676126, 0.2414598 ,
       0.50180816, 0.29299268, 0.6947536 , 0.24459387, 0.82033268,
       0.39193844, 0.90599346, 0.26442957, 0.32150234, 0.3220128 ,
       0.70502976, 0.69623972, 0.26579672, 0.77584327, 0.24541121,
       0.24490372, 0.07551158, 0.26087263, 0.76176924, 0.29640101,
       0.25772789, 0.31539593, 0.88408057, 0.4387068 , 0.59547

In [56]:
#pickling is to save in disk,so that we can transport it later
import pickle

In [57]:
with open('model','wb') as file:
    pickle.dump(reg,file)

In [58]:
with open('scaler','wb') as file:
    pickle.dump(absenteesim_scaler, file)